In [2]:
pip install -r requirements.txt

  Using cached streamlit-1.37.0-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_core-0.2.24-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached kubernetes-30.1.0-py2.py3-none-any.whl.metadata (1.5 kB

In [4]:
import streamlit as st
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
url = "https://intervalrain.github.io/resume2024.pdf"
loader = WebBaseLoader(url)
loader.requests_kwargs = {'verify': False}
docs = loader.load()
docs 

/Users/rainhu/anaconda3/envs/openai/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'intervalrain.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'source': 'https://intervalrain.github.io/resume2024.pdf'}, page_content='%PDF-1.5\n%����\n9 0 obj\n<>\nstream\nx��\x19�n�F�}���\x02\x01\x07д��敧�-E��J\x1cK�`�ڇ\x16�5j�ǘ��\n��>ٜC���aa\x0b3,vU�}M�%��\x08��A��\x7f\x14�5<������y���ח��\x1f\x00�"BS\x1c\\�\x04�8\x0f(J�8�\\�\'��D�X�,\tߎ��^�\x13N�A\x1c�"�\x07n�,B4����p������EJCVu���u�D�ʶ\x0629\x0e\x7f�\x1f$�\n\x12/SR,)!\x12B��E��L�/\x08\x0eY�a�խ�>l��_��W᭫�Z\x0c�\x12<^#�*nO/�\x08e�\x14V~����\x04K\x0c��E�$)��,H�\x02\x15Y\x1e�\x0c�<\x0e:\x1e�\x04�lӰ:D\x05QP�x�\t�3�F1܃\x01\x14�=\x18�\x06q�������m��8FE�h\x1b\x15(�S`>E)�J�\x17c]�\x05���a��1�\x10)�$�W�^�\x04E�\x04u@�\x04����T�\x05���P�� (�s�\x049�!)R�c8��;�ө��O[�lˁA!Ta���1\x19\x12\\�d؏!y�c8��;����=\x19��{Ծ�$~�-)��V�"�Pe�w\x10\x1c\x04��-p\x16�-p\x1er\ri\'@\'!8L�\x07\r\x1f�\x0bκ^C�\x1b\r���,��M\'xSr\rT�\x0fĆ[C��(�f5�C��3��\x1f�� F�����Ю\x1f�\x03�\x1fpm�]�z\rf����N��i�\x16\r�,��f��9Rޒ�B�\x0c�X�:��b\x10m�W\x1a\x02\x1c�\x10C�Q\'�E{3H�T\x07������⎄��\t}Q��\x08s\x0eI\t�\x18oX9��h�W�����B\x

In [12]:
docs

[Document(metadata={'source': 'https://intervalrain.github.io/resume2024.pdf'}, page_content='%PDF-1.5\n%����\n9 0 obj\n<>\nstream\nx��\x19�n�F�}���\x02\x01\x07д��敧�-E��J\x1cK�`�ڇ\x16�5j�ǘ��\n��>ٜC���aa\x0b3,vU�}M�%��\x08��A��\x7f\x14�5<������y���ח��\x1f\x00�"BS\x1c\\�\x04�8\x0f(J�8�\\�\'��D�X�,\tߎ��^�\x13N�A\x1c�"�\x07n�,B4����p������EJCVu���u�D�ʶ\x0629\x0e\x7f�\x1f$�\n\x12/SR,)!\x12B��E��L�/\x08\x0eY�a�խ�>l��_��W᭫�Z\x0c�\x12<^#�*nO/�\x08e�\x14V~����\x04K\x0c��E�$)��,H�\x02\x15Y\x1e�\x0c�<\x0e:\x1e�\x04�lӰ:D\x05QP�x�\t�3�F1܃\x01\x14�=\x18�\x06q�������m��8FE�h\x1b\x15(�S`>E)�J�\x17c]�\x05���a��1�\x10)�$�W�^�\x04E�\x04u@�\x04����T�\x05���P�� (�s�\x049�!)R�c8��;�ө��O[�lˁA!Ta���1\x19\x12\\�d؏!y�c8��;����=\x19��{Ծ�$~�-)��V�"�Pe�w\x10\x1c\x04��-p\x16�-p\x1er\ri\'@\'!8L�\x07\r\x1f�\x0bκ^C�\x1b\r���,��M\'xSr\rT�\x0fĆ[C��(�f5�C��3��\x1f�� F�����Ю\x1f�\x03�\x1fpm�]�z\rf����N��i�\x16\r�,��f��9Rޒ�B�\x0c�X�:��b\x10m�W\x1a\x02\x1c�\x10C�Q\'�E{3H�T\x07������⎄��\t}Q��\x08s\x0eI\t�\x18oX9��h�W�����B\x